Notebook prepared by Henrique Lopes Cardoso (hlc@fe.up.pt).

# REGULARIZATION AND SGD

Regularization is a technique that allows us to avoid overfitting by penalizing excessive feature weights. Several classifiers, such as [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html),  include the option for choosing which regularization term to use.

In this notebook we'll explore the usage of different regularization terms. For that, we'll use a restaurant reviews classification task.

In [2]:
# Loading the data

import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

print(dataset['Liked'].value_counts())
dataset.head()

1    500
0    500
Name: Liked, dtype: int64


,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
# Cleaning the text

import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

corpus = []
ps = PorterStemmer()
for i in range(0,1000):
    # get review and remove non alpha chars
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    # to lower-case and tokenize
    review = review.lower().split()
    # stemming and stop word removal
    review = ' '.join([ps.stem(w) for w in review if not w in set(stopwords.words('english'))])
    corpus.append(review)

In [5]:
# Creating a bag-of-words model

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1500)
X = vectorizer.fit_transform(corpus).toarray()
y = dataset['Liked']

print(X.shape, y.shape)

(1000, 1500) (1000,)


In [6]:
# Splitting the dataset into training and test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(y_train.value_counts())
print(y_test.value_counts())

(800, 1500) (800,)
(200, 1500) (200,)
1    400
0    400
Name: Liked, dtype: int64
0    100
1    100
Name: Liked, dtype: int64


## Logistic Regression

Scikit-learn's [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) includes both L1 and L2 regularizations. L2 is the default.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

clf = LogisticRegression(penalty='l2') # l2 regularization is the default
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

[[83 17]
 [22 78]]


Print the feature weights that we've obtained.

In [15]:
# your code here
for i,v in enumerate(clf.coef_[0]):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.41687
Feature: 1, Score: 0.17698
Feature: 2, Score: 0.00000
Feature: 3, Score: 0.03804
Feature: 4, Score: 0.06701
Feature: 5, Score: 0.35323
Feature: 6, Score: -0.19054
Feature: 7, Score: -0.19417
Feature: 8, Score: -0.05248
Feature: 9, Score: 0.23516
Feature: 10, Score: 0.12999
Feature: 11, Score: 0.21524
Feature: 12, Score: 0.21725
Feature: 13, Score: 0.10537
Feature: 14, Score: 0.01180
Feature: 15, Score: -0.22252
Feature: 16, Score: -0.21240
Feature: 17, Score: 0.14415
Feature: 18, Score: -0.13778
Feature: 19, Score: -0.19425
Feature: 20, Score: -0.33805
Feature: 21, Score: 0.23513
Feature: 22, Score: -0.11946
Feature: 23, Score: -0.39310
Feature: 24, Score: 0.61470
Feature: 25, Score: -0.08571
Feature: 26, Score: 0.40191
Feature: 27, Score: 1.67341
Feature: 28, Score: 0.41871
Feature: 29, Score: 0.15228
Feature: 30, Score: 0.24509
Feature: 31, Score: 0.15231
Feature: 32, Score: 0.15199
Feature: 33, Score: -0.22220
Feature: 34, Score: -0.07387
Feature: 35, Scor

How many features are actually being used? (I.e., how many non-zero weights are there?)

In [17]:
# your code here
num_f = 0
for i,v in enumerate(clf.coef_[0]):
    if v != 0:
        num_f+=1
        print('Feature: %0d, Score: %.5f' % (i,v))

print("Num of features: ", num_f)

Feature: 0, Score: 0.41687
Feature: 1, Score: 0.17698
Feature: 3, Score: 0.03804
Feature: 4, Score: 0.06701
Feature: 5, Score: 0.35323
Feature: 6, Score: -0.19054
Feature: 7, Score: -0.19417
Feature: 8, Score: -0.05248
Feature: 9, Score: 0.23516
Feature: 10, Score: 0.12999
Feature: 11, Score: 0.21524
Feature: 12, Score: 0.21725
Feature: 13, Score: 0.10537
Feature: 14, Score: 0.01180
Feature: 15, Score: -0.22252
Feature: 16, Score: -0.21240
Feature: 17, Score: 0.14415
Feature: 18, Score: -0.13778
Feature: 19, Score: -0.19425
Feature: 20, Score: -0.33805
Feature: 21, Score: 0.23513
Feature: 22, Score: -0.11946
Feature: 23, Score: -0.39310
Feature: 24, Score: 0.61470
Feature: 25, Score: -0.08571
Feature: 26, Score: 0.40191
Feature: 27, Score: 1.67341
Feature: 28, Score: 0.41871
Feature: 29, Score: 0.15228
Feature: 30, Score: 0.24509
Feature: 31, Score: 0.15231
Feature: 32, Score: 0.15199
Feature: 33, Score: -0.22220
Feature: 34, Score: -0.07387
Feature: 35, Score: -0.49750
Feature: 36, Sc

L1 regularization typically obtains sparser weight vectors. Try using L1 regularization (check the documentation for additional changes you might need). How many non-zero weights do you have now?

In [21]:
# your code here

clf = LogisticRegression(penalty='l1', solver='liblinear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

num_f = 0
for i,v in enumerate(clf.coef_[0]):
    if v != 0:
        num_f+=1
        print('Feature: %0d, Score: %.5f' % (i,v))

print("Num of features: ", num_f)

[[89 11]
 [30 70]]
Feature: 24, Score: 0.51547
Feature: 27, Score: 2.12267
Feature: 35, Score: -0.58528
Feature: 40, Score: -0.12348
Feature: 59, Score: -0.34742
Feature: 62, Score: 0.52745
Feature: 72, Score: -0.80347
Feature: 73, Score: -0.22467
Feature: 75, Score: 2.27396
Feature: 83, Score: -0.17914
Feature: 85, Score: -1.73099
Feature: 94, Score: -0.23557
Feature: 108, Score: 1.09803
Feature: 111, Score: 0.53921
Feature: 118, Score: 0.92847
Feature: 119, Score: -0.30396
Feature: 132, Score: -1.12455
Feature: 151, Score: 0.01790
Feature: 153, Score: 0.39358
Feature: 156, Score: -0.22972
Feature: 166, Score: 0.38235
Feature: 174, Score: -0.92642
Feature: 196, Score: 0.17929
Feature: 222, Score: 0.03383
Feature: 247, Score: -0.60715
Feature: 253, Score: 0.43119
Feature: 317, Score: 0.16375
Feature: 320, Score: 0.50697
Feature: 329, Score: 0.50271
Feature: 334, Score: 0.50058
Feature: 339, Score: 3.18598
Feature: 341, Score: 1.38054
Feature: 355, Score: 0.28948
Feature: 362, Score: -0

You can also try using a mix of L1 and L2 (check the documentation for how to do it).

In [29]:
# your code here


clf = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

num_f = 0
for i,v in enumerate(clf.coef_[0]):
    if v != 0:
        num_f+=1
        print('Feature: %0d, Score: %.5f' % (i,v))

print("Num of features: ", num_f)

[[84 16]
 [28 72]]
Feature: 0, Score: 0.10879
Feature: 5, Score: 0.05121
Feature: 20, Score: -0.03213
Feature: 21, Score: 0.01994
Feature: 24, Score: 0.57498
Feature: 26, Score: 0.21775
Feature: 27, Score: 1.93902
Feature: 28, Score: 0.10377
Feature: 35, Score: -0.46768
Feature: 36, Score: -0.02951
Feature: 39, Score: 0.25709
Feature: 40, Score: -0.52795
Feature: 41, Score: -0.19849
Feature: 45, Score: -0.02018
Feature: 59, Score: -0.34287
Feature: 60, Score: 0.39148
Feature: 62, Score: 0.66481
Feature: 66, Score: 0.33264
Feature: 67, Score: -0.02509
Feature: 69, Score: 0.22907
Feature: 72, Score: -0.87996
Feature: 73, Score: -0.59128
Feature: 74, Score: -0.02906
Feature: 75, Score: 1.77625
Feature: 78, Score: 0.11374
Feature: 83, Score: -0.23304
Feature: 84, Score: 0.24899
Feature: 85, Score: -1.53824
Feature: 93, Score: 0.05836
Feature: 94, Score: -0.49893
Feature: 95, Score: 0.02869
Feature: 99, Score: -0.36446
Feature: 108, Score: 0.92868
Feature: 111, Score: 0.80506
Feature: 114, 

C:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## SVM

Scikit-learn's [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) also includes both L1 and L2 regularizations. L2 is the default.

In [30]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

clf = LinearSVC(penalty='l2') # l2 regularization is the default

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

[[82 18]
 [20 80]]


How many features are actually being used? (I.e., how many non-zero weights are there?)

In [35]:
# your code here
num_f = 0
for i,v in enumerate(clf.coef_[0]):
    if v != 0:
        num_f+=1
        print('Feature: %0d, Score: %.5f' % (i,v))

print("Num of features: ", num_f)

Feature: 0, Score: 0.17146
Feature: 1, Score: 0.04973
Feature: 5, Score: 0.38568
Feature: 6, Score: -0.15287
Feature: 7, Score: -0.19968
Feature: 9, Score: 0.22866
Feature: 10, Score: 0.05657
Feature: 11, Score: 0.07275
Feature: 12, Score: 0.22575
Feature: 14, Score: 0.08033
Feature: 15, Score: -0.24371
Feature: 16, Score: -0.26253
Feature: 18, Score: -0.16223
Feature: 19, Score: -0.13497
Feature: 20, Score: -0.33221
Feature: 21, Score: 0.19787
Feature: 22, Score: -0.18496
Feature: 23, Score: -0.47013
Feature: 24, Score: 0.30388
Feature: 25, Score: -0.00000
Feature: 26, Score: 0.32420
Feature: 27, Score: 0.99828
Feature: 28, Score: 0.44897
Feature: 30, Score: 0.17032
Feature: 31, Score: 0.16742
Feature: 32, Score: 0.04432
Feature: 33, Score: -0.14198
Feature: 35, Score: -0.20338
Feature: 36, Score: -0.64093
Feature: 37, Score: -0.31153
Feature: 38, Score: -0.14065
Feature: 39, Score: 0.71098
Feature: 40, Score: -0.64529
Feature: 41, Score: -0.65804
Feature: 42, Score: -0.03000
Feature:

Try using L1 regularization (check the documentation for additional changes you might need). How many non-zero weights do you have now?

In [38]:
# your code here


clf = LinearSVC(penalty='l1', loss='squared_hinge', dual=False) # l2 regularization is the default
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

num_f = 0
for i,v in enumerate(clf.coef_[0]):
    if v != 0:
        num_f+=1
        print('Feature: %0d, Score: %.5f' % (i,v))

print("Num of features: ", num_f)

[[86 14]
 [27 73]]
Feature: 5, Score: 0.59674
Feature: 21, Score: 0.00656
Feature: 24, Score: 0.23531
Feature: 26, Score: 0.24446
Feature: 27, Score: 1.38139
Feature: 28, Score: 0.13405
Feature: 39, Score: 1.28966
Feature: 40, Score: -0.70526
Feature: 41, Score: -0.79303
Feature: 45, Score: -0.18553
Feature: 56, Score: 0.06736
Feature: 57, Score: 0.18340
Feature: 60, Score: 1.28900
Feature: 62, Score: 0.25395
Feature: 66, Score: 0.45282
Feature: 67, Score: -0.09833
Feature: 69, Score: 0.19887
Feature: 72, Score: -1.05121
Feature: 73, Score: -0.39991
Feature: 74, Score: -0.20721
Feature: 75, Score: 1.38783
Feature: 78, Score: 0.41784
Feature: 79, Score: 0.00563
Feature: 83, Score: -0.09442
Feature: 84, Score: 0.90297
Feature: 85, Score: -1.06448
Feature: 93, Score: 0.11774
Feature: 95, Score: 0.20880
Feature: 99, Score: -0.55237
Feature: 106, Score: 0.49315
Feature: 108, Score: 0.96514
Feature: 111, Score: 0.93323
Feature: 115, Score: -0.53635
Feature: 118, Score: 0.40419
Feature: 119, 

## SGD Classifier

Scikit-learn's [SGD Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) implements regularized linear models (such as SVM and Logistic Regression) with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing learning rate.

Several loss functions can be used, namely *hinge loss* (which corresponds to SVM) and *log loss* (which corresponds to Logistic Regression). And as before, you can use L1 and/or L2 regularization.

The *max_iter* parameter allows you to set the maximum number of epochs, where an epoch corresponds to going through the whole dataset for training. Also, *learning_rate* allows you to set a learning rate schedule.

Several parameters allow you to define stopping criteria: *tol* specifies a tolerance loss value or stopping criterion, while *n_iter_no_change* indicates the number of iterations with no improvement that should be observed before stopping; *early_stopping* allows us to use a validation set (a fraction *validation_fraction* of the training data) on which the stopping criterion will be checked (instead of checking the loss on the training data).

The *verbose* parameter allows you to set a verbosity (output) level.

Try using SGD, and explore different parameters!

In [45]:
# your code here
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SGDClassifier(loss='modified_huber',max_iter=1000, tol=1e-3, n_iter_no_change=500, early_stopping=True, verbose=2))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))

-- Epoch 1
Norm: 12516.64, NNZs: 1311, Bias: -3873.833683, T: 720, Avg. loss: 6197.796824
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 9834.22, NNZs: 1311, Bias: -4004.824279, T: 1440, Avg. loss: 1114.794600
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 7886.61, NNZs: 1311, Bias: -3664.934160, T: 2160, Avg. loss: 186.640440
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 6795.15, NNZs: 1311, Bias: -3360.316155, T: 2880, Avg. loss: 122.566260
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 6158.88, NNZs: 1311, Bias: -3033.344816, T: 3600, Avg. loss: 119.324391
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 5690.06, NNZs: 1311, Bias: -2712.518904, T: 4320, Avg. loss: 132.385635
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 5176.14, NNZs: 1311, Bias: -2559.476853, T: 5040, Avg. loss: 43.099759
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 4771.13, NNZs: 1311, Bias: -2427.617444, T: 5760, Avg. loss: 45.705838
Total training time: 0.02 seconds.
-- Epoch

-- Epoch 120
Norm: 891.58, NNZs: 1311, Bias: -898.066368, T: 86400, Avg. loss: 0.181107
Total training time: 0.36 seconds.
-- Epoch 121
Norm: 888.43, NNZs: 1311, Bias: -894.603979, T: 87120, Avg. loss: 0.119821
Total training time: 0.37 seconds.
-- Epoch 122
Norm: 884.35, NNZs: 1311, Bias: -891.996165, T: 87840, Avg. loss: 0.226607
Total training time: 0.37 seconds.
-- Epoch 123
Norm: 879.14, NNZs: 1311, Bias: -890.158025, T: 88560, Avg. loss: 0.088648
Total training time: 0.37 seconds.
-- Epoch 124
Norm: 874.07, NNZs: 1311, Bias: -888.484226, T: 89280, Avg. loss: 0.080823
Total training time: 0.38 seconds.
-- Epoch 125
Norm: 870.23, NNZs: 1311, Bias: -885.761480, T: 90000, Avg. loss: 0.164639
Total training time: 0.38 seconds.
-- Epoch 126
Norm: 866.11, NNZs: 1311, Bias: -883.628044, T: 90720, Avg. loss: 0.185964
Total training time: 0.38 seconds.
-- Epoch 127
Norm: 862.99, NNZs: 1311, Bias: -880.340097, T: 91440, Avg. loss: 0.166648
Total training time: 0.39 seconds.
-- Epoch 128
Nor

Norm: 630.24, NNZs: 1311, Bias: -692.152354, T: 172080, Avg. loss: 0.027941
Total training time: 0.74 seconds.
-- Epoch 240
Norm: 628.09, NNZs: 1311, Bias: -691.730079, T: 172800, Avg. loss: 0.030220
Total training time: 0.74 seconds.
-- Epoch 241
Norm: 626.21, NNZs: 1311, Bias: -691.090489, T: 173520, Avg. loss: 0.018710
Total training time: 0.74 seconds.
-- Epoch 242
Norm: 623.67, NNZs: 1311, Bias: -691.060670, T: 174240, Avg. loss: 0.011841
Total training time: 0.75 seconds.
-- Epoch 243
Norm: 624.47, NNZs: 1311, Bias: -688.369954, T: 174960, Avg. loss: 0.142904
Total training time: 0.75 seconds.
-- Epoch 244
Norm: 624.29, NNZs: 1311, Bias: -686.514351, T: 175680, Avg. loss: 0.146364
Total training time: 0.75 seconds.
-- Epoch 245
Norm: 622.50, NNZs: 1311, Bias: -685.865966, T: 176400, Avg. loss: 0.041221
Total training time: 0.76 seconds.
-- Epoch 246
Norm: 620.46, NNZs: 1311, Bias: -685.457602, T: 177120, Avg. loss: 0.028249
Total training time: 0.76 seconds.
-- Epoch 247
Norm: 62

-- Epoch 365
Norm: 516.49, NNZs: 1311, Bias: -589.070034, T: 262800, Avg. loss: 0.020914
Total training time: 1.12 seconds.
-- Epoch 366
Norm: 515.79, NNZs: 1311, Bias: -588.460266, T: 263520, Avg. loss: 0.032355
Total training time: 1.13 seconds.
-- Epoch 367
Norm: 514.68, NNZs: 1311, Bias: -588.203127, T: 264240, Avg. loss: 0.007908
Total training time: 1.13 seconds.
-- Epoch 368
Norm: 513.63, NNZs: 1311, Bias: -587.912359, T: 264960, Avg. loss: 0.017554
Total training time: 1.13 seconds.
-- Epoch 369
Norm: 512.43, NNZs: 1311, Bias: -587.751208, T: 265680, Avg. loss: 0.009028
Total training time: 1.14 seconds.
-- Epoch 370
Norm: 511.49, NNZs: 1311, Bias: -587.372943, T: 266400, Avg. loss: 0.019307
Total training time: 1.14 seconds.
-- Epoch 371
Norm: 510.49, NNZs: 1311, Bias: -587.059245, T: 267120, Avg. loss: 0.018431
Total training time: 1.14 seconds.
-- Epoch 372
Norm: 510.29, NNZs: 1311, Bias: -586.117939, T: 267840, Avg. loss: 0.041022
Total training time: 1.14 seconds.
-- Epoch

Norm: 454.94, NNZs: 1311, Bias: -528.583448, T: 348480, Avg. loss: 0.013895
Total training time: 1.49 seconds.
-- Epoch 485
Norm: 454.65, NNZs: 1311, Bias: -528.075509, T: 349200, Avg. loss: 0.020451
Total training time: 1.49 seconds.
-- Epoch 486
Norm: 454.56, NNZs: 1311, Bias: -527.427445, T: 349920, Avg. loss: 0.047077
Total training time: 1.49 seconds.
-- Epoch 487
Norm: 454.25, NNZs: 1311, Bias: -526.902978, T: 350640, Avg. loss: 0.017463
Total training time: 1.50 seconds.
-- Epoch 488
Norm: 454.02, NNZs: 1311, Bias: -526.337278, T: 351360, Avg. loss: 0.026558
Total training time: 1.50 seconds.
-- Epoch 489
Norm: 453.31, NNZs: 1311, Bias: -526.158599, T: 352080, Avg. loss: 0.019073
Total training time: 1.50 seconds.
-- Epoch 490
Norm: 452.53, NNZs: 1311, Bias: -526.031990, T: 352800, Avg. loss: 0.032707
Total training time: 1.51 seconds.
-- Epoch 491
Norm: 452.15, NNZs: 1311, Bias: -525.579368, T: 353520, Avg. loss: 0.013183
Total training time: 1.51 seconds.
-- Epoch 492
Norm: 45

Stochastic gradient descent updates the model weights base on one example at a time. Instead, we can compute the gradient over batches of training instances before updating the weights.

SGDClassifier allows us to do so via [*partial_fit*](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier.partial_fit), which corresponds to training the model with a specific set of examples for a single epoch. To properly use this method, we need to split our data into mini-batches and then iterate through them for as many epochs as we want.
Matters such as objective convergence, early stopping, and learning rate adjustments must be handled manually.

Try it out!

In [55]:
# your code here
import random
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # apply same transformation to test data


def batch(iterable_X, iterable_y, n=1):
    l = len(iterable_X)
    for ndx in range(0, l, n):
        yield iterable_X[ndx:min(ndx + n, l)], iterable_y[ndx:min(ndx + n, l)]


clf = SGDClassifier(alpha=.0001, loss='log', penalty='l2', n_jobs=-1, shuffle=True, max_iter=100, verbose=0, tol=0.001)
 
ROUNDS = 6
for _ in range(ROUNDS):
    batcherator = batch(X_train, y_train, 10)
    for index, (chunk_X, chunk_y) in enumerate(batcherator):
        clf.partial_fit(chunk_X, chunk_y, classes=[0, 1, 2])
 
        y_predicted = clf.predict(X_test)
        print(accuracy_score(y_test, y_predicted))

0.28
0.33
0.325
0.365
0.375
0.41
0.43
0.435
0.445
0.455
0.45
0.455
0.44
0.44
0.455
0.465
0.475
0.48
0.485
0.46
0.495
0.495
0.485
0.505
0.51
0.52
0.51
0.51
0.525
0.535
0.545
0.565
0.585
0.605
0.59
0.6
0.615
0.625
0.645
0.63
0.61
0.625
0.63
0.625
0.625
0.615
0.605
0.6
0.61
0.625
0.625
0.625
0.635
0.65
0.65
0.645
0.635
0.64
0.665
0.665
0.665
0.665
0.665
0.665
0.685
0.695
0.705
0.705
0.715
0.72
0.72
0.725
0.725
0.725
0.72
0.725
0.715
0.715
0.725
0.74
0.735
0.745
0.735
0.735
0.735
0.735
0.735
0.74
0.74
0.735
0.73
0.735
0.735
0.74
0.73
0.73
0.725
0.72
0.71
0.715
0.72
0.72
0.725
0.73
0.735
0.735
0.735
0.735
0.73
0.73
0.725
0.72
0.74
0.735
0.745
0.74
0.74
0.75
0.755
0.75
0.745
0.745
0.745
0.745
0.745
0.75
0.75
0.745
0.74
0.745
0.75
0.75
0.745
0.745
0.745
0.745
0.745
0.745
0.745
0.745
0.745
0.745
0.745
0.73
0.73
0.73
0.73
0.73
0.725
0.73
0.735
0.735
0.735
0.735
0.73
0.725
0.715
0.715
0.72
0.725
0.715
0.725
0.73
0.73
0.73
0.73
0.73
0.73
0.73
0.725
0.73
0.73
0.73
0.73
0.73
0.73
0.735
0.735
0.72
0